In [1]:
import re
import string

class Processor:
    """
    Una clase hecha para limpiar el texto que traen los tweets.
    
    El metodo process limpia el texto haciendo uso de los demas metodos
    """
    
    def __init__(self):
        self.dirtyReps = re.compile(r'([^lL0])\1{1,}')
        self.dirtySpaces = re.compile(r'(\.|,|:|;|!|\?|\[|\]|\(|\))[A-Za-z0-9]+')
        self.dirtyK = re.compile('[^o]k')
        self.dirtyJaja = re.compile(r'[ja]{5,}')
        self.dirtyJeje = re.compile(r'[je]{5,}')
        self.uglySeparator = 'THIS-IS-A-SEPARATOR'
        self.urlPattern=re.compile(r'(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?\xab\xbb\u201c\u201d\u2018\u2019]))')
    
    def replaceAccents(self,word):
        """ Reemplaza las tildes """
        word = word.replace('í','i')
        word = word.replace('ó','o')
        word = word.replace('ò','o')
        word = word.replace('ñ','n')
        word = word.replace('é','e')
        word = word.replace('è','e')
        word = word.replace('á','a')
        word = word.replace('à','a')
        word = word.replace('ü','u')
        word = word.replace('ú','u')
        word = word.replace('ö','o')
        word = word.replace('ë','e')
        word = word.replace('ï','i')
        return word

    def processDetails(self,word):
        """ 
        Clase 'comodin' para simplificar el procesamiento
        
        Por ejemplo: re -> muy, 100% -> muy, #hashtag -> hashtag
        
        Se puede mejorar haciendo un diccionario de reemplazo
        word es toda la frase> word = "esto es una 100% frase" -> "esto es una muy frase"
        """
        word = word.replace('#', '')
        word = word.replace(' re ',' muy ')
        word = word.replace(' 100% ', ' muy ')
        word = word.replace('mercado libre', ''.join(['mercado','libre']))
        word = word.replace('mercado pago', ''.join(['mercado','pago']))
        word = word.replace(' x ',' por ')
        word = word.replace(' q ', ' que ')
        word = word.replace(' qu ', ' que ')
        word = word.replace(' qeh ', ' que ')
        word = word.replace(' qhe ', ' que ')
        word = word.replace(' qe ', ' que ')
        word = word.replace(' ke ', ' que ')
        word = word.replace(' keh ', ' que ')
        word = word.replace(' khe ', ' que ')
        word = word.replace(' k ', ' que ')
        word = word.replace('lll','ll')
        word = word.replace('la puta madre','lpm')
        word = word.replace('la puta que te pario','lpqtp')
        word = word.replace('la concha de la lora','lcdll')
        word = word.replace('hijo de puta', 'hdp')
        
        return word

    def processJaja(self,word):
        """
        jajajaj, jajja, jajaj, jejejaja, jejej, etc -> jaja
        """
        while self.dirtyJaja.search(word)!=None:
            word = word.replace(self.dirtyJaja.search(word).group(),'jaja')
        while self.dirtyJeje.search(word)!=None:
            word = word.replace(self.dirtyJeje.search(word).group(),'jaja')
        return word    
    
    def processRep(self,word):
        """
        Saca las repeticiones por ej: buenasss, bueeena, essss -> buenas, buena, es
        No remueve la doble l
        """
        while self.dirtyReps.search(word)!=None:
            word = word.replace(self.dirtyReps.search(word).group(),
                                self.dirtyReps.search(word).group()[0])
        return word

    def processSpaces(self,word):
        """
        Remueve espacios de sobra, incluidos los de adelante de la sentencia y los del final
        """
        while self.dirtySpaces.search(word)!=None:
            word = word.replace(self.dirtySpaces.search(word).group()[0],
                                self.dirtySpaces.search(word).group()[0]+' ')
        word = word.lstrip().rstrip()
        return word
    
    def processK(self,word):
        """
        Remueve la k en palabras como kilombo -> quilombo
        """
        while self.dirtyK.search(word)!=None:
            word = word.replace(self.dirtyK.search(word).group(),
                                self.dirtyK.search(word).group()[0]+'qu')
        return word
    
    def processNumbers(self,word):
        """
        Los numero normalmente no tienen peso en una frase, los remueve
        Excepto por la frase "esta todo de 10" -> "esta todo de diez"
        """
        word = word.replace('10',' diez')
        word = re.sub(r'\d+', '', word)  
        return word
    
    def removePuntuation(self, word):
        """
        Remueve todas las puntuaciones
        """
        word = "".join([char for char in word if char not in string.punctuation]) 
        return word
    
    def removeLinks(self, word):
        """
        Elimina los links y deja solo 'url' como constancia de un link eliminado
        Puede servir para encontrar noticias neutras
        """
        return self.urlPattern.sub(' url ', word)
    
    def removeUsers(self, word):
        """
        Elimina los users para no sesgar la clasificacion
        """
        return re.sub('@[^\s]+','',word)
    
    def process(self,x, lematizar=False):
        """
        Metodo que usa todos los otros metodos para limpiar el texto
        """
        if len(x) > 0:
            str(x).replace('\r','').replace('\n','')
            x = self.removeLinks(x)
            x = self.removeUsers(x)
            x = self.replaceAccents(x.lower())  
            x = self.removePuntuation(x)
            x = self.processNumbers(x)
            x = self.processRep(x)
            x = self.processJaja(x)
            x = self.processSpaces(x)
            x = self.processDetails(x)
            x = self.processK(x)
            if lematizar:
                # spacy lematizar
                pass
        return x

In [2]:
import pandas as pd

dataset = pd.read_csv(r'/home/gastondg/Documentos/IA/datasets/posta/clasificado.csv',
                      encoding='utf-8')
dataset.sample(10)

,text,label
2371,"Hola, soy Cinthia. Te respondí por DM, seguimo...",NEUTRAL
2270,"Hola Gaston, estamos trabajando en la zona:htt...",NEUTRAL
2739,Easy POS SOFTWARE Facturacion ELECTRONICA AFIP...,NEUTRAL
1460,Ja ja y doble ja le hice el reclamo por el 080...,NEGATIVE
2623,"Hola, pasame por DM tu DNI y contame a qué hac...",NEUTRAL
2785,"Hola Jesica, gracias por comentarnos lo sucedi...",NEUTRAL
591,Que emoción!!!,POSITIVE
2559,"Hola, Luis te respondí por DM para ayudarte. P...",NEUTRAL
1497,No es Aerolineas...es la mierda de la campora ...,NEGATIVE
1614,"Sí que la cierre, se ahorra plata y problemas",NEGATIVE


In [3]:
limpieza = Processor()
limpieza.process("Haber si esto funciona www.github.com ESTAMOSSSS PROBando")

'haber si esto funciona url estamos probando'

In [4]:
"""
Limpiamos los datos
"""
dataset_limpio = dataset.copy()
dataset_limpio['text'] = dataset_limpio['text'].apply(limpieza.process)
dataset_limpio.head()

,text,label
0,cuando desde se ponen la diez url,POSITIVE
1,que alegria juan disfruta a lo grande aguardam...,POSITIVE
2,hola gaton te respondimos por dm continuamos p...,POSITIVE
3,buenas pudiste solucionarlo gracias,POSITIVE
4,estaria bueno que respondan,POSITIVE


In [5]:
"""
Separamos datos en entrenamiento y pruebas
"""
from sklearn.model_selection import train_test_split

x = dataset_limpio['text']
y = dataset_limpio['label']
X_train, X_test, y_train, y_test = train_test_split(x, y,test_size= 0.2, random_state=4)

print('Numero total de filas en el dataset: {}'.format(dataset_limpio.shape[0]))
print('Numero de filas en el set de entrenamiento: {}'.format(X_train.shape[0]))
print('Numero de filas en el set de testing: {}'.format(X_test.shape[0]))

Numero total de filas en el dataset: 3135
Numero de filas en el set de entrenamiento: 2508
Numero de filas en el set de testing: 627


In [6]:
"""
Vectorizamos
"""
from sklearn.feature_extraction.text import TfidfVectorizer

# Instanciamos el vectorizador
vectorizer = TfidfVectorizer(ngram_range=(1, 5)# Tomamos frases de hasta 6 palabras
                                 )

# fit and transform on it the training features
#word_vectorizer.fit(X_train)
training_data = vectorizer.fit_transform(X_train)

#transform the test features to sparse matrix
testing_data = vectorizer.transform(X_test)


In [7]:
"""
Clasificacion
Corremos una regresion logistica en cada label separada
"""
from sklearn.naive_bayes import MultinomialNB

naive_bayes = MultinomialNB(alpha=0.005)
naive_bayes.fit(training_data, y_train)

MultinomialNB(alpha=0.005, class_prior=None, fit_prior=True)

In [8]:
predictions = naive_bayes.predict(testing_data)

In [9]:
"""
Accuracy Tests
Hacemos las pruebas de precision del modelo entrenado
"""

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
print('Precision score: ', format(precision_score(y_test, predictions,average='macro')))
print('Recall score: ', format(recall_score(y_test, predictions,average='macro')))
print('F1 score: ', format(f1_score(y_test, predictions,average='macro')))


Accuracy score:  0.7001594896331739
Precision score:  0.7104698686192111
Recall score:  0.6955246913580247
F1 score:  0.6956126060565357


In [145]:
"""
Lo guardamos en un pickle para utilizarlo sin volver a entrenarlo
"""

import pickle

clasificador = 'clasificador_multinomial_v1.pkl'
vectorizador = 'vectorizador_tfidf_v1.pkl'

pickle.dump(naive_bayes, open( clasificador, "wb" ))
pickle.dump(vectorizer, open(vectorizador, "wb" ))

In [147]:
with open(clasificador, 'rb') as m1:
    clasif = pickle.load(m1,encoding='latin1')
    
with open(vectorizador, 'rb') as m2:
    vect = pickle.load(m2,encoding='latin1')

In [150]:
print(clasif.predict(vect.transform(['es una verga'])))

NEGATIVE


In [144]:
print(naive_bayes.predict(vectorizer.transform(['es una verga'])))
print(naive_bayes.predict(vectorizer.transform(['sos un groso hermano'])))
print(naive_bayes.predict(vectorizer.transform(['muy rico todo'])))
print(naive_bayes.predict(vectorizer.transform(['son unos chorros de mierda'])))
print(naive_bayes.predict(vectorizer.transform(['prefiero una rica torta'])))
print(naive_bayes.predict(vectorizer.transform(['es todo culpa de ellos'])))
print(naive_bayes.predict(vectorizer.transform(['me gusta jugar al tenis'])))

['NEGATIVE']
['POSITIVE']
['POSITIVE']
['NEGATIVE']
['POSITIVE']
['NEGATIVE']
['POSITIVE']
